**File Number Renaming Ascending Order**

In [ ]:
import os
import shutil

dataset_folder_path = '/content/DS1'
xml_path = '/content/DS1 XML'

new_ds = '/content/DS1 new'
new_ds_xml = '/content/DS1 XML new'

ctr = 1

for filename in os.listdir(dataset_folder_path):
    if os.path.isfile(os.path.join(dataset_folder_path, filename)):

        jpg = 'DS1-' + str(ctr) + '.jpg'
        xml = 'DS1-' + str(ctr) + '.xml'
        ctr+=1

        src_path = os.path.join(dataset_folder_path, filename)
        dest_path = os.path.join(new_ds, jpg)
        shutil.copy(src_path, dest_path)

        src_path2 = os.path.join(xml_path, filename.replace('.jpg', '.xml'))
        dest_path2 = os.path.join(new_ds_xml, xml)
        shutil.copy(src_path2, dest_path2)
        print(filename, filename.replace('.jpg', '.xml'))

**Zip Folder Downloading**

In [ ]:
import shutil
folder_to_download = '/content/DS4 XML'

zip_file_name = 'DS4 XML.zip'

shutil.make_archive(zip_file_name.split('.')[0], 'zip', folder_to_download)

**Processing Annotations**

In [ ]:
import os
# os.makedirs('/content/DS2 XML', exist_ok=True)
# os.makedirs('/content/DS3 XML', exist_ok=True)
os.makedirs('/content/DS4 XML', exist_ok=True)

Renaming XML Object Name

In [ ]:
import xml.etree.ElementTree as ET
folder  = '/content/drive/MyDrive/Raw Image dataset/DS4 XML'
save_folder = '/content/DS4 XML'

small = 0
medium = 0
large = 0

for xml in os.listdir(folder):
  source_xml_path = os.path.join(folder, xml)
  destination_xml_path = os.path.join(save_folder, xml)

  tree = ET.parse(source_xml_path)
  root = tree.getroot()

  size = root.find('size')
  w = int(size.find('width').text)
  h = int(size.find('height').text)

  for obj in root.findall('object'):
    name = obj.find('name')
    box = obj.find('bndbox')

    width = float(box.find('xmax').text) - float(box.find('xmin').text)
    height = float(box.find('ymax').text) - float(box.find('ymin').text)

    width = width * 300 / w
    height = height * 300 / h

    area = width * height

    if(area <= 1024):
      name.text = 'Small Pothole'
      small += 1
    elif(area > 1024 and area <= 9216):
      name.text = 'Medium Pothole'
      medium += 1
    elif(area > 9216):
      name.text = 'Large Pothole'
      large += 1

  new_xml_file = os.path.join(save_folder, os.path.basename(xml))
  tree.write(new_xml_file)

print(small, medium, large)

125 498 447


Single File Test

In [ ]:
tree = ET.parse(folder + '/img-1.xml')
root = tree.getroot()

size = root.find('size')
w = int(size.find('width').text)
h = int(size.find('height').text)

for obj in root.findall('object'):
  name = obj.find('name')
  box = obj.find('bndbox')

  width = float(box.find('xmax').text) - float(box.find('xmin').text)
  height = float(box.find('ymax').text) - float(box.find('ymin').text)

  width = width * 300 / w
  height = height * 300 / h

  area = width * height

  if(area <= 1024):
    name.text = 'Small Pothole'
  elif(area > 1024 and area <= 9216):
    name.text = 'Medium Pothole'
  elif(area > 9216):
    name.text = 'Large Pothole'

  print(name.text)
  print(area)
  print()

Medium Pothole
7475.0



Showing Image with Box

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

def parse_xml_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    boxes = []
    for obj in root.findall('object'):
        pothole_size = obj.find('name').text
        box = obj.find('bndbox')
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        boxes.append((xmin, ymin, xmax, ymax, pothole_size))

    return boxes

def draw_boxes_on_image(image_path, boxes, small, medium, large):
    image = cv2.imread(image_path)
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    for box in boxes:
        xmin, ymin, xmax, ymax, pothole_size = box

        color = (0, 255, 0)
        if(pothole_size == 'Small Pothole'):
          color = (0, 255, 0)
          small += 1
        elif(pothole_size == 'Medium Pothole'):
          color = (255, 255, 0)
          medium += 1
        elif(pothole_size == 'Large Pothole'):
          color = (255, 0, 0)
          large += 1

        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 2)

    return image, small, medium, large

images_folder = '/content/drive/MyDrive/Raw Image dataset/DS2'

annotations_folder = '/content/drive/MyDrive/Raw Image dataset/DS2 XML'

small = 0
medium = 0
large = 0

for image_file in tqdm(os.listdir(images_folder)):
  if image_file.endswith(('.jpg', '.jpeg', '.png')):
    image_path = os.path.join(images_folder, image_file)
    annotation_file = os.path.join(annotations_folder, image_file.replace('.jpg', '.xml'))

    if os.path.exists(annotation_file):
      boxes = parse_xml_annotation(annotation_file)
      annotated_image, small, medium, large = draw_boxes_on_image(image_path, boxes, small, medium, large)

      fig, axes = plt.subplots(1, 2, figsize=(10, 5))

      axes[0].imshow(annotated_image)
      axes[0].set_title(image_path[-30:])

      axes[0].axis('off')

      plt.show()

print(small, medium, large)

100%|██████████| 656/656 [04:19<00:00,  2.52it/s]

625 812 285


Single File Test

In [ ]:
image_path = '/content/drive/MyDrive/Pothole_img_xml/img-2.jpg'
annotation_file = '/content/drive/MyDrive/Pothole_img_xml/img-2.xml'

if os.path.exists(annotation_file):
    boxes = parse_xml_annotation(annotation_file)
    annotated_image = draw_boxes_on_image(image_path, boxes)

    print(image_path)
    plt.imshow(annotated_image)